# Single Subject Segmentation Q1K

In [ ]:
# import packages
#import q1k_sync_tools as qst
import mne
import mne_bids
#mne.viz.set_browser_backend('qt')
#%matplotlib qt
import pandas as pd
pd.options.mode.chained_assignment = None
import pylossless as ll
import numpy as np
import glob

In [ ]:
project_path = "/home/james/q1k/pilot/q1k-external-pilot/"
derivative_path = "derivatives/pylossless"
task_id_in = "mn"
task_id_out = "mn"
subject_id = '004'
session_id = '01'

bids_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id_out, run="1", datatype="eeg", root=project_path + derivative_path
)
print(bids_path)

In [ ]:
# Read the BIDS pylossless output file..
eeg_raw = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)

In [ ]:
# Get the events form the annotations
eeg_events, eeg_event_dict  = mne.events_from_annotations(eeg_raw)

In [ ]:
eeg_events

In [ ]:
eeg_event_dict

In [ ]:
# Plot channel data only
mne.viz.plot_raw(eeg_raw, events=eeg_events, event_id=eeg_event_dict)

In [ ]:
# set file variables
#subject_raw_eeg = glob.glob('sourcedata/020_1/020_1_go_*.mff')
subject_raw_eeg = 'sourcedata/012_1/012_1_eeg/012_vp_20230417_125533.mff'
subject_raw_et = 'sourcedata/012_1/012_1_eyetracking/012_vp_1.asc'
subject_derivative = 'output_vp/derivatives/pylossless/sub-012/eeg/sub-012_task-vp_eeg.edf'
task_code = 'vp'

In [ ]:
subject_raw_eeg

In [ ]:
# Load EEG recording
eeg_raw = mne.io.read_raw_egi(subject_raw_eeg)

# Get event info from source recording
eeg_events = mne.find_events(eeg_raw, shortest_event=1)
eeg_event_dict = qst.get_event_dict(eeg_raw, eeg_events)



# Below is for syncing EEG and ET sessions then combining the data.

# Infer new events and labels from EEG paradigm procedure
#eeg_events, eeg_stims, eeg_iti, eeg_event_dict = qst.eeg_event_test(eeg_events, eeg_event_dict, task_name=task_code)

# Read eyetracking data and extact events
#et_raw = mne.io.read_raw_eyelink(subject_raw_et)
#et_annot_events, et_annot_event_dict = mne.events_from_annotations(et_raw)
#et_raw_df = et_raw.to_data_frame()

# Infer new events from paradigm procedure
#et_event_raw_df, et_events, et_stims, et_iti = qst.et_event_test(et_raw_df, task_name=task_code)

# Meld both ET and EEG together into new raw structure
#eeg_et_raw = qst.eeg_et_combine(eeg_raw, et_raw, eeg_stims, et_stims)

# Using the melded raw, build final event objects
#eeg_et_events = mne.find_events(eeg_et_raw, shortest_event=1)
#eeg_et_event_dict = qst.get_event_dict(eeg_et_raw, eeg_et_events)

# Free up memory
#del et_event_raw_df
#del et_events
#del et_stims
#del et_iti
#del et_raw_df
#del et_annot_events
#del et_annot_event_dict
#del eeg_events
#del eeg_stims
#del eeg_iti
#del eeg_event_dict

In [ ]:
eeg_events

In [ ]:
eeg_event_dict

In [ ]:
# Plot channel data only
mne.viz.plot_raw(eeg_raw, events=eeg_events, event_id=eeg_event_dict)

In [ ]:
# Build initial state; nothing applied yet
ll_state = ll.LosslessPipeline()
ll_state = ll_state.load_ll_derivative(subject_derivative)

In [ ]:
# Plot channel data only
mne.viz.plot_raw(ll_state.raw, events=eeg_events, event_id=eeg_event_dict)

In [ ]:
# Plot comonents
ll_state.ica2.plot_sources(ll_state.raw)


In [ ]:
# Plot topos
ll_state.ica2.plot_components(inst=ll_state.raw)

In [ ]:
# Merge marks down to bads (aka manual)
ll_qcr = ll_state.raw.copy()
manual = []
for flag_type in ll_state.flags['ch']:
    manual.extend(ll_state.flags['ch'][flag_type])
print(ll_state.flags['ch'])
ll_qcr.info['bads'].extend(manual)
ll_qcr.info['bads'] = list(set(ll_qcr.info['bads']))

# Read the ICLabel info from file and add to exclude
df = pd.read_csv(subject_derivative.replace('_eeg.edf', '_iclabels.tsv'), sep='\t')
ll_state.ica2.exclude = list(df[df['ic_type'].str.match('eog|muscle|ch_noise|ecg')].index)

# Load the data and apply the ICA
ll_qcr.load_data()
ll_state.ica2.apply(ll_qcr)
ll_qcr = ll_qcr.filter(l_freq=1.0, h_freq=40.0)
ll_qcr = ll_qcr.set_eeg_reference(ref_channels="average")
ll_qcr

In [ ]:
# Plot cleaned data
mne.viz.plot_raw(ll_qcr, events=eeg_events, event_id=eeg_event_dict)

In [ ]:
# Take a look at the event dictionairy
eeg_event_dict

In [ ]:
# Relabel condition vars for niceness
eeg_event_dict['fq06hz/target/disp/dstr'] = eeg_event_dict.pop('dstr')
eeg_event_dict['fq15hz/target/disp/fvct'] = eeg_event_dict.pop('fvct')

In [ ]:
# Epoch structure is created for ALL events, then you 'segment' by indexing into it
epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1, tmax=2.0, on_missing='warn', event_repeated='drop')

In [ ]:
# Check trial counts
epochs[['fq06hz', 'fq15hz']]

In [ ]:
# Plot ERPs
evokeds = {'fq06hz': epochs['fq06hz'].average(), 'fq15hz': epochs['fq15hz'].average()}
mne.viz.plot_compare_evokeds(evokeds, picks=['E70'], combine='mean')

In [ ]:
mne.write_evokeds(f'erp_fif_files/vp/009_1_vp_ave.fif',list(evokeds.values()), overwrite=True)

In [ ]:
eeg_events

In [ ]:
mask = np.isin(eeg_events[:,2],[eeg_event_dict['fq15hz/target/disp/fvct']])
eeg_events = eeg_events[mask]


epochs = mne.Epochs(ll_qcr, eeg_events, event_id=eeg_event_dict, tmin=-1.0, tmax=2.0, on_missing='warn', event_repeated='merge')

freqs = np.arange(5,50,2)
n_cycles = freqs / 2.0  # different number of cycle per frequency
power, itc = mne.time_frequency.tfr_morlet(
    epochs,
    freqs=freqs,
    n_cycles=n_cycles,
    use_fft=True,
    return_itc=True,
    decim=3,
    n_jobs=None,
)

In [ ]:
power

In [ ]:
power.plot(65,baseline=(-0.5, 0), mode="logratio", title='power')

In [ ]:
itc.plot()